<div style="border: 2px solid #575757; padding: 10px; border-radius: 5px; background-color: #e1e1e1; color: black; text-align: center;">
  <h1 style="margin: 0;">Cleaning & Extraction of OpenModelica model outputs</h1>
</div>

<div style="border: 2px solid #FFA500; padding: 10px; border-radius: 5px; background-color: #FFFACD; color: black; text-align: center;">
  <h2 style="margin: 0;">Libraries importation</h2>
</div>

**Import required libraries:**

In [1]:
import os
import pandas as pd
from tqdm.notebook import tqdm

<div style="border: 2px solid #FFA500; padding: 10px; border-radius: 5px; background-color: #FFFACD; color: black; text-align: center;">
  <h2 style="margin: 0;">Cleaning of OM outputs</h2>
</div>

**Reads all the original exported results of the OpenModelica model to clean them**
- OpenModelica exports seems to often lead to duplicates of certain rows
- The exports are located in the *OM_original_outputs* folder
- The script cleans the files by suppressing them

**Optional if you already have the .csv files in the "*OM_cleaned_ouputs*" folder**

In [ ]:
# Define input and output folders
input_folder = "OM_original_outputs"
output_folder = "OM_cleaned_outputs"
os.makedirs(output_folder, exist_ok=True)

# List all CSV files in the input folder
csv_files = [f for f in os.listdir(input_folder) if f.endswith(".csv")]

# Process each file with progress tracking
for file_name in tqdm(csv_files, desc="Cleaning CSV files"):
    input_file = os.path.join(input_folder, file_name)
    output_file = os.path.join(output_folder, file_name)

    try:
        df = pd.read_csv(input_file)
        time_column = df.columns[0]

        # Filter valid time values (0 to 31536000 in 3600-second steps)
        valid_times = set(range(0, 31536001, 3600))
        df_filtered = df[df[time_column].isin(valid_times)]
        df_filtered = df_filtered.drop_duplicates(subset=[time_column])

        # Save the cleaned CSV
        df_filtered.to_csv(output_file, index=False)
    except Exception as e:
        print(f"❌ Error cleaning {file_name}: {e}")
    else:
        print(f"✅ Cleaned data saved to: {output_file}")

<div style="border: 2px solid #FFA500; padding: 10px; border-radius: 5px; background-color: #FFFACD; color: black; text-align: center;">
  <h2 style="margin: 0;">Combined Electricity, H<sub>2</sub>, and Heat data extraction</h2>
</div>

**Reads all the cleaned results of the OpenModelica model to extract and classify them for further processing**
- The "*Balance of flows*" Jupyter Notebook uses separate electricity, hydrogen, and heat files for simplicity
- The datafiles are located in the *OM_cleaned_outputs* folder
- The script extract relevant variables in the files and concatenate them into 3 relevant files for further processing

**Optional if you already have the .xlsx files in the "*OM_extracted_ouputs*" folder**

In [2]:
# Define input and output paths
input_folder = "OM_cleaned_outputs"
output_folder = "OM_extracted_outputs"
os.makedirs(output_folder, exist_ok=True)

# Prepare file list
csv_files = [f for f in os.listdir(input_folder) if f.endswith(".csv")]

# Initialize containers
time_column = None
electricity_data = {}
electricity_load_data = []
hydrogen_data = {}
heat_data = {}
heat_load_data = []

# Define relevant hydrogen and additional variables
hydrogen_extra_columns = [
    "results_electrical.Ppemel",
    "results_electrical.Pcom",
    "results_heating.m_dot_H2",
    "results_heating.Pfuel"
]

# Process files with progress bar
for file in tqdm(csv_files, desc="Processing CSV files"):
    file_path = os.path.join(input_folder, file)
    try:
        df = pd.read_csv(file_path)

        # Extract time column once
        if time_column is None and 'time' in df.columns:
            time_column = df[['time']].copy()

        # ELECTRICITY
        elec_cols = [col for col in df.columns if col.startswith("results_electrical.P")]
        for col in elec_cols:
            category = col.split(".")[1]
            electricity_data.setdefault(category, []).append(df[[col]].rename(columns={col: file}))

        if "data_Input.electricity_load" in df.columns:
            df_load = df[["data_Input.electricity_load"]].copy()
            df_load.columns = [file]
            electricity_load_data.append(df_load)

        # HYDROGEN
        hydrogen_cols = [col for col in df.columns if col.startswith("results_hydrogen.")]
        additional_cols = [col for col in hydrogen_extra_columns if col in df.columns]
        for col in hydrogen_cols + additional_cols:
            if "." not in col:
                print(f"Skipping malformed column: {col}")
                continue

            category = col.split(".")[1]
            variable = col.split(".")[2] if len(col.split(".")) > 2 else category
            if col == "results_heating.m_dot_H2":
                variable += "_heating"

            hydrogen_data.setdefault(variable, []).append(df[[col]].rename(columns={col: file}))

        # HEAT
        heat_cols = [col for col in df.columns if col.startswith("results_heating.P") or col.startswith("results_heating.Q")]
        for col in heat_cols:
            category = col.split(".")[1]
            heat_data.setdefault(category, []).append(df[[col]].rename(columns={col: file}))

        if "data_Input.heat_load" in df.columns:
            df_load = df[["data_Input.heat_load"]].copy()
            df_load.columns = [file]
            heat_load_data.append(df_load)

    except Exception as e:
        print(f"Error processing {file}: {e}")

# Helper to write grouped data to Excel
def write_to_excel(filepath, data_dict, extra_data=None, extra_sheet_name=None, label=""):
    print(f"Saving {label} data to {filepath}...")
    with pd.ExcelWriter(filepath) as writer:
        for category, dfs in tqdm(data_dict.items(), desc=f"Writing {label} sheets"):
            df_final = pd.concat([time_column] + dfs, axis=1) if time_column is not None else pd.concat(dfs, axis=1)
            df_final.to_excel(writer, sheet_name=category[:31], index=False)  # Excel sheet name max = 31 chars
        if extra_data:
            df_extra = pd.concat([time_column] + extra_data, axis=1) if time_column is not None else pd.concat(extra_data, axis=1)
            df_extra.to_excel(writer, sheet_name=extra_sheet_name, index=False)

# Save electricity
write_to_excel(
    os.path.join(output_folder, "elec_variables.xlsx"),
    electricity_data,
    electricity_load_data,
    "electricity_load",
    label="electricity"
)

# Save hydrogen
write_to_excel(
    os.path.join(output_folder, "h2_variables.xlsx"),
    hydrogen_data,
    label="hydrogen"
)

# Save heat
write_to_excel(
    os.path.join(output_folder, "heat_variables.xlsx"),
    heat_data,
    heat_load_data,
    "heat_load",
    label="heat"
)

print("✅ All results successfully saved to OM_extracted_outputs.")

Processing CSV files:   0%|          | 0/28 [00:00<?, ?it/s]

Saving electricity data to OM_extracted_outputs\elec_variables.xlsx...


Writing electricity sheets:   0%|          | 0/11 [00:00<?, ?it/s]

Saving hydrogen data to OM_extracted_outputs\h2_variables.xlsx...


Writing hydrogen sheets:   0%|          | 0/23 [00:00<?, ?it/s]

Saving heat data to OM_extracted_outputs\heat_variables.xlsx...


Writing heat sheets:   0%|          | 0/7 [00:00<?, ?it/s]

✅ All results successfully saved to OM_extracted_outputs.


<div style="border: 2px solid #FFA500; padding: 10px; border-radius: 5px; background-color: #FFFACD; color: black; text-align: center;">
  <h2 style="margin: 0;">Separated Electricity, H<sub>2</sub>, and Heat data extraction</h2>
</div>

**Reads all the cleaned results of the OpenModelica model to extract and classify them for further processing**
- The "*Balance of flows*" Jupyter Notebook uses separate electricity, hydrogen, and heat files for simplicity
- The precedent script is here separated into 3 independant scripts for flexibility in use

**Optional if you already have the .xlsx files in the "*OM_extracted_ouputs*" folder**

<div style="border: 2px solid rgba(0, 158, 115, 1); padding: 10px; border-radius: 5px; background-color: rgba(0, 158, 115, 0.3); color: black; text-align: center;">
  <h2 style="margin: 0;">Electricity data extraction</h2>
</div>

In [3]:
# Define input and output paths
input_folder = "OM_cleaned_outputs"
output_folder = "OM_extracted_outputs"
os.makedirs(output_folder, exist_ok=True)
output_file = os.path.join(output_folder, "elec_variables.xlsx")

# List all CSV files in the input folder
csv_files = [f for f in os.listdir(input_folder) if f.endswith(".csv")]

# Initialize storage
data_dict = {}
time_column = None
electricity_load_data = []

# Process each CSV file
for file in csv_files:
    file_path = os.path.join(input_folder, file)
    try:
        df = pd.read_csv(file_path)

        # Extract 'time' column once
        if time_column is None and 'time' in df.columns:
            time_column = df[['time']].copy()

        # Identify electrical-related columns
        electrical_columns = [col for col in df.columns if col.startswith("results_electrical.P")]

        # Extract electricity load column if present
        if "data_Input.electricity_load" in df.columns:
            df_load = df[["data_Input.electricity_load"]].copy()
            df_load.columns = [file]
            electricity_load_data.append(df_load)

        # Store electrical data by category
        for col in electrical_columns:
            category = col.split(".")[1]
            if category not in data_dict:
                data_dict[category] = []
            df_col = df[[col]].copy()
            df_col.columns = [file]
            data_dict[category].append(df_col)

    except Exception as e:
        print(f"Error processing {file}: {e}")

# Save to Excel
with pd.ExcelWriter(output_file) as writer:
    for category, dfs in data_dict.items():
        df_final = pd.concat([time_column] + dfs, axis=1)
        df_final.to_excel(writer, sheet_name=category, index=False)

    if electricity_load_data:
        df_load_final = pd.concat([time_column] + electricity_load_data, axis=1)
        df_load_final.to_excel(writer, sheet_name="electricity_load", index=False)

print(f"Results saved to {output_file}")

Results saved to OM_extracted_outputs\elec_variables.xlsx


<div style="border: 2px solid rgba(0, 116, 130, 1); padding: 10px; border-radius: 5px; background-color: rgba(0, 116, 130, 0.2); color: black; text-align: center;">
  <h2 style="margin: 0;">H<sub>2</sub> data extraction</h2>
</div>

In [4]:
# Define input and output paths
input_folder = "OM_cleaned_outputs"
output_folder = "OM_extracted_outputs"
os.makedirs(output_folder, exist_ok=True)
output_file = os.path.join(output_folder, "h2_variables.xlsx")

# List all CSV files in the input folder
csv_files = [f for f in os.listdir(input_folder) if f.endswith(".csv")]

# Initialize storage
data_dict = {}
time_column = None

# Process each CSV file
for file in csv_files:
    file_path = os.path.join(input_folder, file)
    try:
        df = pd.read_csv(file_path)

        # Extract 'time' column once
        if time_column is None and 'time' in df.columns:
            time_column = df[['time']].copy()

        # Identify hydrogen-related and specified additional columns
        hydrogen_columns = [col for col in df.columns if col.startswith("results_hydrogen.")]
        additional_columns = [
            "results_electrical.Ppemel",
            "results_electrical.Pcom",
            "results_heating.m_dot_H2",
            "results_heating.Pfuel"
        ]
        relevant_columns = [col for col in hydrogen_columns + additional_columns if col in df.columns]

        for col in relevant_columns:
            if "." not in col:
                print(f"Skipping malformed column: {col}")
                continue

            category = col.split(".")[1]
            variable = col.split(".")[2] if len(col.split(".")) > 2 else category

            if col == "results_heating.m_dot_H2":
                variable += "_heating"

            if variable not in data_dict:
                data_dict[variable] = []

            df_col = df[[col]].copy()
            df_col.columns = [file]
            data_dict[variable].append(df_col)

    except Exception as e:
        print(f"Error processing {file}: {e}")

# Save to Excel
with pd.ExcelWriter(output_file) as writer:
    for category, dfs in data_dict.items():
        if time_column is not None:
            df_final = pd.concat([time_column] + dfs, axis=1)
        else:
            df_final = pd.concat(dfs, axis=1)
        df_final.to_excel(writer, sheet_name=category, index=False)

print(f"Results saved to {output_file}")

Results saved to OM_extracted_outputs\h2_variables.xlsx


<div style="border: 2px solid #9f8189; padding: 10px; border-radius: 5px; background-color: #ffcad4; color: black; text-align: center;">
  <h2 style="margin: 0;">Heat data extraction</h2>
</div>

In [5]:
# Define input and output paths
input_folder = "OM_cleaned_outputs"
output_folder = "OM_extracted_outputs"
os.makedirs(output_folder, exist_ok=True)
output_file = os.path.join(output_folder, "heat_variables.xlsx")

# List all CSV files in the input folder
csv_files = [f for f in os.listdir(input_folder) if f.endswith(".csv")]

# Initialize storage
data_dict = {}
time_column = None
heat_load_data = []

# Process each CSV file
for file in csv_files:
    file_path = os.path.join(input_folder, file)
    try:
        df = pd.read_csv(file_path)

        # Extract 'time' column once
        if time_column is None and 'time' in df.columns:
            time_column = df[['time']].copy()

        # Identify heat-related columns
        heat_columns = [col for col in df.columns if col.startswith("results_heating.P") or col.startswith("results_heating.Q")]

        # Extract heat load column if present
        if "data_Input.heat_load" in df.columns:
            df_load = df[["data_Input.heat_load"]].copy()
            df_load.columns = [file]
            heat_load_data.append(df_load)

        # Store heat data by category
        for col in heat_columns:
            category = col.split(".")[1]
            if category not in data_dict:
                data_dict[category] = []
            df_col = df[[col]].copy()
            df_col.columns = [file]
            data_dict[category].append(df_col)

    except Exception as e:
        print(f"Error processing {file}: {e}")

# Save to Excel
with pd.ExcelWriter(output_file) as writer:
    for category, dfs in data_dict.items():
        df_final = pd.concat([time_column] + dfs, axis=1)
        df_final.to_excel(writer, sheet_name=category, index=False)

    if heat_load_data:
        df_load_final = pd.concat([time_column] + heat_load_data, axis=1)
        df_load_final.to_excel(writer, sheet_name="heat_load", index=False)

print(f"Results saved to {output_file}")

Results saved to OM_extracted_outputs\heat_variables.xlsx
